In [ ]:
using Plots
import Plots: px
theme(:default)
ENV["LINES"] = 20

In [ ]:
include("parser.jl")
include("models.jl")
gpu()
download(SIGDataSet)

In [ ]:
data  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-train-high"));
vocab = Vocabulary(data)
edata = encode(data,vocab);
test  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-test"));
tdata = encode(test,vocab);
dictionary = [parseDataLine(line) for line in  eachline("./data/unimorph/spa/spa") if line != ""];

In [ ]:
vocab.chars.toElement;

In [ ]:
vocab.specialIndices

In [150]:
trainsfs    = unique(map(x->join(x.surface),data));
testsfs     = unique(map(x->join(x.surface),test)); 
dictsfs     = unique([map(x->join(x.surface),dictionary); trainsfs; testsfs]);
unseensfs   = [x for x in dictsfs if x ∉ trainsfs]

314523-element Array{String,1}:
 "ababilladas"  
 "ababillada"   
 "ababillados"  
 "ababillado"   
 "ababillándose"
 "ababillaos"   
 "ababillarse"  
 "ababíllate"   
 ⋮              
 "zuzase"       
 "zuzasteis"    
 "zuzaste"      
 "zuzas"        
 "zuza"         
 "zuzo"         
 "zuzó"         

In [ ]:
morph = EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph, data, vocab; optim=Adam(), epoch=50)

In [ ]:
morphv =  EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morphv.encoder, morph.encoder)
copytoparams(morphv.Wμ, morph.Wμ)
copytoparams(morphv.Wσa, morph.Wσa)
copytoparams(morphv.Wμa, morph.Wμa)
copytoparams(morphv.Wσ, morph.Wσ)
copytoparams(morphv.Weaμ, morph.Weaμ)
copytoparams(morphv.Weaσ, morph.Weaσ)
morphv.decoder.embedding = morphv.encoder.embedding
train_vae!(morphv, data, vocab; optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

In [ ]:
samples = sample(morphv, vocab, edata; N=10000);

In [ ]:
unique(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
unique(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
unique(length(samples[findall([s ∈ dictsfs for s in samples])]))

In [ ]:
sampleinter(morphv, vocab, data; N=10)

In [ ]:
au, sigma, mu = calc_au(morphv,tdata; delta=0.01)

In [ ]:
mi = calc_mi(morphv,tdata)

In [ ]:
calc_ppl(morphv,tdata)

In [ ]:
atts = attentions(morph,edata, vocab);

In [ ]:
function getsingle(sfs,exs,perms, αs,i) 
    join(vocab.chars[sfs[i]]), map(e->join(vocab.chars[e]),exs[i][perms[i]]), map(x->x[:,perms[i],i],αs)
end

In [ ]:
i = rand(1:length(atts))
s1,e1,α1 =  getsingle(atts[i]...,rand(1:16))
ys = 1:512
h1 = heatmap(e1, ys, α1.αu; size=(1200,800), xtickfont = font(10, "Halvetica"), title= s1 * ", mu", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);
h2 = heatmap(e1, ys, α1.ασ ;size=(1200,800), xtickfont = font(10, "Halvetica"), title=s1 * ", sigma", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);

In [ ]:
println(s1);println(e1)

In [ ]:
h1

# To-Dos
1. Add unrelated surface forms to the examplars too see whether the attention mappings are meaningful
2. Comparing with Normal Vae
3. Different Sampling Function on LM ($top_k$, temperature)
4. Attention in Decoder
5. Attention to the hidden state sequence, not just to the final state.
6. Metric for testing generation quality?

In [89]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [187]:
morph3=nothing; morphv=nothing; morph=nothing; morph2=nothing; KnetLayers.gc();

In [188]:
morph2 = VAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph2, data, vocab; optim=Adam(), B=16, epoch=20);

(loss = 23.89124849700928,)
(loss = 17.421337057495116,)
(loss = 13.838453059387207,)
(loss = 10.603875322723388,)
(loss = 8.237073970031739,)
(loss = 6.580452914047242,)
(loss = 5.3860104988098145,)
(loss = 4.392002590942383,)
(loss = 3.683862651062012,)
(loss = 3.033609958076477,)
(loss = 2.5571276865005492,)
(loss = 2.208719219493866,)
(loss = 1.9281249282836914,)
(loss = 1.6575271893501282,)
(loss = 1.5428996180057526,)
(loss = 1.3619286948204041,)
(loss = 1.2220081511497498,)
(loss = 1.0673871292114259,)
(loss = 0.9783469723701477,)
(loss = 0.9391327674388885,)


In [189]:
morph3 = VAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morph3.encoder, morph2.encoder)
copytoparams(morph3.Wμ, morph2.Wμ)
copytoparams(morph3.Wσ, morph2.Wσ)
copytoparams(morph3.dec_embed, morph3.encoder.embedding)
train_vae!(morph3, data, vocab; B=16, optim=Adam(lr=0.002), epoch=40, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=4)

(kl_weight = 0.1f0, fbr = 4, loss = 14.659807389831544)
(kl_weight = 0.2f0, fbr = 4, loss = 13.541754844665528)
(kl_weight = 0.3f0, fbr = 4, loss = 14.147867942810059)
(kl_weight = 0.4f0, fbr = 4, loss = 15.002387745666503)
(kl_weight = 0.5f0, fbr = 4, loss = 15.724966891479491)
(kl_weight = 0.6f0, fbr = 4, loss = 16.409736642456053)
(kl_weight = 0.70000005f0, fbr = 4, loss = 16.94357787628174)
(kl_weight = 0.8000001f0, fbr = 4, loss = 17.363033013916017)
(kl_weight = 0.9000001f0, fbr = 4, loss = 17.690947436523437)
(kl_weight = 1.0f0, fbr = 4, loss = 17.880223318481445)
(kl_weight = 1.0f0, fbr = 4, loss = 18.165674900817873)
(kl_weight = 1.0f0, fbr = 4, loss = 17.82644891204834)
(kl_weight = 1.0f0, fbr = 4, loss = 17.57117961425781)
(kl_weight = 1.0f0, fbr = 4, loss = 17.255397506713866)
(kl_weight = 1.0f0, fbr = 4, loss = 17.02019940032959)
(kl_weight = 1.0f0, fbr = 4, loss = 16.723417462158203)
(kl_weight = 1.0f0, fbr = 4, loss = 16.504892237854005)
(kl_weight = 1.0f0, fbr = 4, loss

In [190]:
samples = sample(morph3, vocab, edata; N=10000, useprior=true)

10005-element Array{String,1}:
 "pespuntarais"   
 "os acartonarais"
 "enterneceré"    
 "aplacan"        
 "relajabais"     
 "contundiré"     
 "reinvirtierais" 
 "rehuisteis"     
 ⋮                
 "despeinaste"    
 "tensionabas"    
 "recularen"      
 "se ciaría"      
 "camelados"      
 "devoínes"       
 "no intuseis"    

In [191]:
samples[findall([s ∈ trainsfs for s in samples])]

1161-element Array{String,1}:
 "enterneceré"   
 "contundiré"    
 "reinvirtierais"
 "rehuisteis"    
 "apila"         
 "amasije"       
 "desligarían"   
 "descocería"    
 ⋮               
 "reinstaurare"  
 "regrabáremos"  
 "dividía"       
 "distorsione"   
 "despeinaste"   
 "recularen"     
 "camelados"     

In [192]:
samples[findall([s ∈ testsfs for s in samples])]

6-element Array{String,1}:
 "okupara"      
 "no enamore"   
 "situarías"    
 "vibrase"      
 "no deleguemos"
 "refundamos"   

In [193]:
samples[findall([s ∈ dictsfs for s in samples])]

3932-element Array{String,1}:
 "pespuntarais"   
 "os acartonarais"
 "enterneceré"    
 "aplacan"        
 "relajabais"     
 "contundiré"     
 "reinvirtierais" 
 "rehuisteis"     
 ⋮                
 "alambicaban"    
 "recluir"        
 "distorsione"    
 "despeinaste"    
 "tensionabas"    
 "recularen"      
 "camelados"      

In [194]:
samples[findall([s ∈ unseensfs for s in samples])] 

2771-element Array{String,1}:
 "pespuntarais"   
 "os acartonarais"
 "aplacan"        
 "relajabais"     
 "repensado"      
 "aludieren"      
 "adujarías"      
 "recelaren"      
 ⋮                
 "romanceásemos"  
 "encajares"      
 "mentalizaras"   
 "conjurarais"    
 "alambicaban"    
 "recluir"        
 "tensionabas"    

In [195]:
inters = sampleinter(morph3, vocab, data; N=10)

12-element Array{String,1}:
 "no desencajen"  
 "no desencajen"  
 "no desencajen"  
 "no desencajen"  
 "no desenchufe"  
 "no descontrolen"
 "constreñís"     
 "constelare"     
 "chismearon"     
 "chismearon"     
 "chismear"       
 "chismear"       

In [196]:
findall([s ∈ dictsfs for s in inters])

12-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [197]:
au, sigma, mu = calc_au(morph3, tdata; delta=0.01)

(16, Float32[0.39809954; 0.4430144; … ; 0.32374555; 0.38953856], K32(16,1)[0.05120561⋯])

In [198]:
mi =  calc_mi(morph3,tdata)

4.94630160522461

In [ ]:
calc_ppl(morph3, edata; nsample=500, B=16)

In [138]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [ ]:
model = LSTM_LM(length(vocab.chars); H=768, E=16)

In [ ]:
train_rnnlm!(model, data, vocab; epoch=40, optim=Adam(), B=16)

In [ ]:
 calc_ppllm(model, tdata)

In [ ]:
samples = sample(model, vocab; N=10000)

In [ ]:
samples[findall([s ∈ trainsfs for s in samples])]

In [ ]:
samples[findall([s ∈ testsfs for s in samples])]

In [ ]:
samples[findall([s ∈ dictsfs for s in samples])]

In [ ]:
 calc_ppllm(model, edata)

In [202]:
?@eval

```
@eval [mod,] ex
```

Evaluate an expression with values interpolated into it using `eval`. If two arguments are provided, the first is the module to evaluate in.
